# 2. Spark SQL

https://drive.google.com/file/d/1_mH86hiGYxRnYZBs_JaPkLcxEHUOjrcr/view 

1. Read data from from S3 s3a://aws-stocks-dataset/AAL.csv
as dataframe

2. Create a new column month, based on Date
3. With Spark SQL get the highest price per month and sort the
output so the latest is first.
4. Print the output

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.master("local[1]").appName("AWSpark").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1. Read data from from S3 s3a://aws-stocks-dataset/AAL.csv as dataframe

In [10]:
df = spark.read.option("header","true").csv("s3a://aws-stocks-dataset/AAL.csv")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|
+----------+------------------+------------------+-------+------------------+------------------+------------------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|             21.75|              21.5|20.268939971923828|
|04-10-2005|21.440000534057617|21.440000534057617|1768800|              

### 2. Create a new column month, based on Date

In [8]:
## not in sql...
# df = df.withColumn('year', split(df['Date'], '-').getItem(0)) \
#        .withColumn('month', split(df['Date'], '-').getItem(1)) \
#       .withColumn('day', split(df['Date'], '-').getItem(2))
# df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df = df.withColumn("Date_dt", to_date(col('Date'), "dd-MM-yyyy"))
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|   Date_dt|
+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|2005-09-27|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|2005-09-28|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|2005-09-29|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|2005-09-30|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|             21.75|              21.5|20.268939971923

In [36]:
# register the table 
df.createOrReplaceTempView("stocks")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_month = spark.sql("SELECT Date_dt, EXTRACT(MONTH FROM Date_dt) AS Month FROM stocks")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_month.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|   Date_dt|Month|
+----------+-----+
|2005-09-27|    9|
|2005-09-28|    9|
|2005-09-29|    9|
|2005-09-30|    9|
|2005-10-03|   10|
+----------+-----+
only showing top 5 rows

In [44]:
spark.sql("SELECT Date_dt, EXTRACT(MONTH FROM Date_dt) AS Month FROM stocks Limit 5").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|   Date_dt|Month|
+----------+-----+
|2005-09-27|    9|
|2005-09-28|    9|
|2005-09-29|    9|
|2005-09-30|    9|
|2005-10-03|   10|
+----------+-----+

In [48]:
df = spark.sql("SELECT *, EXTRACT(MONTH FROM Date_dt) AS Month FROM stocks")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+----+------------------+------------------+----------+-----+
|      Date|               Low|              Open| Volume|High|             Close|    Adjusted Close|   Date_dt|Month|
+----------+------------------+------------------+-------+----+------------------+------------------+----------+-----+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.0|19.299999237060547|18.194910049438477|2005-09-27|    9|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.0|              20.5|19.326204299926758|2005-09-28|    9|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.0|20.209999084472656| 19.05280303955078|2005-09-29|    9|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.0|21.010000228881836|19.806997299194336|2005-09-30|    9|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|21.0|              21.5|20.268939971923828|2005-10-03|   10|
|04-10-2005|21.440000534057617|21.44000053405761

### 3. With Spark SQL get the highest price per month and sort the output so the latest is first.

In [34]:
# changing datatype of a row
# its the same: 
# df = df.withColumn('High', col('High').cast('float'))

df = df.withColumn('High', df.High.cast('float'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- High: float (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adjusted Close: string (nullable = true)
 |-- Date_dt: date (nullable = true)
 |-- Month: integer (nullable = true)

In [51]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+----+------------------+------------------+----------+-----+
|      Date|               Low|              Open| Volume|High|             Close|    Adjusted Close|   Date_dt|Month|
+----------+------------------+------------------+-------+----+------------------+------------------+----------+-----+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.0|19.299999237060547|18.194910049438477|2005-09-27|    9|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.0|              20.5|19.326204299926758|2005-09-28|    9|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.0|20.209999084472656| 19.05280303955078|2005-09-29|    9|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.0|21.010000228881836|19.806997299194336|2005-09-30|    9|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|21.0|              21.5|20.268939971923828|2005-10-03|   10|
+----------+------------------+-----------------

In [69]:
# SQL view must be created again with the full table for the new sql query
df.createOrReplaceTempView('stocks')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
query = """
SELECT Month, max(High) as Highest_Price 
FROM stocks 
GROUP BY Month
ORDER BY Month
"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Print the output

In [73]:
spark.sql(query).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+
|Month|Highest_Price|
+-----+-------------+
|    1|         62.0|
|    2|         59.0|
|    3|         57.0|
|    4|         53.0|
|    5|         52.0|
|    6|         51.0|
|    7|         56.0|
|    8|         52.0|
|    9|         48.0|
|   10|         53.0|
|   11|         63.0|
|   12|         62.0|
+-----+-------------+